In [3]:
ls_columns_output = [
    '№ партии',
    'марка стали',
    'диаметр',
    'толщина стенки',
    'Врем. сопротивление',
    'прогноз Врем. сопротивление',
    'MAE Врем. сопротивление',
    'MSE Врем. сопротивление',
    'Предел текучести',
    'прогноз Предел текучести', 
    'MAE Предел текучести',
    'MSE Предел текучести',
    '1 зона по ВТР закалка',
    '2 зона по ВТР закалка',
    'шаг балок закалочная печь, сек',
    'Скорость прохождения трубы через спрейер, м/с', 
    't˚ C трубы после спреера',
    '1 зона ВТР и уставка отпуск', 
    '2 зона ВТР и уставка отпуск', 
    '3 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск',
    '5 зона ВТР и уставка отпуск',
    'шаг балок отпускная печь, сек',
    'C',
    'Mn',
    'Si',
    'P',
    'S',
    'Cr',
    'Ni',
    'Cu',
    'Al',
    'V',
    'Ti',
    'Nb',
    'Mo',
    'N',
    'C-coef',
    'Параметр закалка',
    'Параметр отпуск',
    'Параметр отпуск новый V'
    ]

In [4]:
import pandas as pd
import numpy as np
import pickle
import json
import import_ipynb
from my_libs.calc_features import *
from sklearn.preprocessing import StandardScaler

database = pd.read_excel('prepared_to_saw.xlsx')

def close_value(df, row, value):
    s_close = df[row].value_counts().index[0]
    min_delta = np.abs(s_close-value)
    for s_ in df[row].value_counts().index:
        delta = np.abs(s_-value)
        if delta<min_delta:
            min_delta = delta
            s_close = s_
    return s_close

def find_close_sort(df):
    print('Исторические данные загружены')
    tmp = pd.DataFrame()
    for i in range(len(df.index)):
        answ = df.iloc[i, :].copy()
        mark = answ['марка стали']
        d = answ['диаметр']
        s = answ['толщина стенки']
        if database[database['марка стали']==mark].shape[0]==0:
            mark = mark.split('-')[0]
            answ = database[database['марка стали']==mark]
        if database[database['диаметр']==d].shape[0]!=0:
            answ = database[database['диаметр']==d]
            s_close = close_value(database, 'толщина стенки',s)
            answ = database[database['толщина стенки']==s_close]
        elif database[database['толщина стенки']==s].shape[0]!=0:
            answ = database[database['толщина стенки']==s]
            d_close = close_value(database, 'диаметр',d)
            answ = database[database['диаметр']==d_close]
        answ = answ[answ['Дата термообработки'] == answ['Дата термообработки'].max()][:3]
        print('Исторические режимы найдены', answ.shape)
        tmp = pd.concat([tmp, answ])
    return tmp 

def find_mark_typesize(df):

    for i in range(len(df.index)):
        answ = df.iloc[i, :].copy()
        mark = answ['марка стали']
        d = answ['диаметр']
        s = answ['толщина стенки']
        if mark!=None:
            answ = database[database['марка стали']==mark]
        if d!=None:
            answ = database[database['диаметр']==d]
        if s!=None:
            answ = database[database['толщина стенки']==s]
        answ = answ[answ['Дата термообработки'] == answ['Дата термообработки'].max()][:3]
        print('Исторические режимы найдены', answ.shape)
        tmp = pd.concat([tmp, answ])
    return tmp

# возвращает модель, список признаков, по которому строилась модель и скейлер
def load_model(dir_name, target):
    model = pickle.load(open(dir_name+'/RF_model_'+target+'.sav', 'rb'))
    ls_need_col = json.load(open(dir_name+'/ls_need_col', "r"))
#     try:
    scaler = StandardScaler()
    scale_data = json.load(open(dir_name+'/scaler', "r"))
    scaler.mean_ = scale_data[0]
    scaler.scale_ = scale_data[1]
#     except:
#         scaler=None
    return model, ls_need_col, scaler


def make_result_valid_file(file_name, dir_names, targets, output_filename):
    """Сохраняет файл с результатами """
    result = pd.DataFrame()
    base = pd.DataFrame()
    for target, dir_name in zip(targets,dir_names):
        model, ls_need_col, scaler = load_model(dir_name, target)
        try:
            valid = clean_data(file_name, ls_need_col+[
                '№ партии', 
                'марка стали'])
        except KeyError:
            valid = clean_data(file_name, ls_need_col)
        print('valid ', valid.columns)
    #     print(valid['C-coef'])
    #     valid = valid[valid['C-coef']<0.6]
        try:
            y_valid = valid[target].copy()
        except KeyError:
            y_valid = [0 for x in range(0,valid.shape[0])]
        try:
            X_valid = valid[ls_need_col+['№ партии', 'марка стали']].copy()
        except KeyError:
            X_valid = valid[ls_need_col].copy()
        X_valid_c = X_valid[ls_need_col].copy()
        X_valid_c = scaler.transform(X_valid_c)
        y_pred = model.predict(X_valid_c)
        result[target] = y_valid
        result['прогноз '+target] = y_pred
        result['MAE '+target] = np.abs(result[target]- result['прогноз '+target])
        result['MSE '+target] = result['MAE '+target].apply(lambda x: x*x)

#     result['отношение'] = result[targets[0]]/result[targets[1]]
    base = find_close_sort(X_valid).drop_duplicates()
    result = pd.concat([result, X_valid], axis=1)
    base = base[list(set(base.columns) & set(result.columns))]
    base.index = [str(i)+' исторический' for i in range(len(base.index))]
    result = pd.concat([result, base])
    result = result[ls_columns_output]
    result.to_excel(output_filename)
    return result, y_valid, y_pred, base

def make_result_valid_file_multi(file_name, dir_names, targets, output_filename):
    """Сохраняет файл с результатами """
    writer = pd.ExcelWriter(output_filename)
    df_dict = pd.read_excel(file_name, sheetname=None)
    for sheet, df in zip(df_dict, df_dict.values()):
        result = pd.DataFrame()
        base = pd.DataFrame()
        k=0
        for target, dir_name in zip(targets,dir_names):
            model, ls_need_col, scaler = load_model(dir_name, target)
            valid = clean_data_sheets(df, ls_need_col+['№ партии', 'марка стали'])
            y_valid = valid[target].copy()
            X_valid = valid[ls_need_col+['№ партии', 'марка стали']].copy()
            X_valid_c = X_valid[ls_need_col].copy()

            y_pred = model.predict(X_valid_c)

            result[target] = y_valid
            result['прогноз '+target] = y_pred
            result['MAE '+target] = np.abs(result[target]- result['прогноз '+target])
            result['MSE '+target] = result['MAE '+target].apply(lambda x: x*x)
            X_valid = new_spr(X_valid)
            k+=1
            if k==2:
                result = pd.concat([result, X_valid], axis=1)
#                 result = pd.concat([result, base])
                result.to_excel(writer, sheet)
                
    writer.save()
    return result, y_valid, y_pred, base

file_name = 'DATA/modes_before_model/127x12,5 37g2f _ input - копия.xlsx'
# target = 'Врем. сопротивление'
targets = ['Предел текучести','Врем. сопротивление']
dir_names = ['DATA/MODELS_RF/YS RF valid', 'DATA/MODELS_RF/H RF valid']
output_filename = 'DATA/modes_after_model/127x12,5 37g2f _ input - копия test find base modes.xlsx'

df, y_valid, y_pred, dbasemode = make_result_valid_file(file_name, dir_names, targets, output_filename)

importing Jupyter notebook from D:\Repo\PycharmProjects\gitPNTZ\PNTZ\my_libs\calc_features.ipynb


D:\Repo\PycharmProjects\gitPNTZ\PNTZ\my_libs\calc_features.ipynb:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {


valid  Index(['1 зона по ВТР закалка', '2 зона по ВТР закалка',
       '3 зона по ВТР закалка',
       'Скорость прохождения трубы через спрейер, м/с',
       't˚ C трубы после спреера', '1 зона ВТР и уставка отпуск',
       '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
       '4 зона ВТР и уставка отпуск', '5 зона ВТР и уставка отпуск',
       'шаг балок закалочная печь, сек', 'шаг балок отпускная печь, сек',
       'диаметр', 'толщина стенки', 'C', 'Mn', 'Si', 'P', 'S', 'Cr', 'Ni',
       'Cu', 'Al', 'V', 'Ti', 'Nb', 'Mo', 'N', 'Параметр закалка',
       'Параметр отпуск новый V', 'C-coef', 'Величина зерна', '№ партии',
       'марка стали', 'Предел текучести', 'Врем. сопротивление'],
      dtype='object')
valid  Index(['1 зона по ВТР закалка', '2 зона по ВТР закалка',
       'Скорость прохождения трубы через спрейер, м/с',
       't˚ C трубы после спреера', '1 зона ВТР и уставка отпуск',
       '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
       '4 з

In [5]:
df

,№ партии,марка стали,диаметр,толщина стенки,Врем. сопротивление,прогноз Врем. сопротивление,MAE Врем. сопротивление,MSE Врем. сопротивление,Предел текучести,прогноз Предел текучести,...,Al,V,Ti,Nb,Mo,N,C-coef,Параметр закалка,Параметр отпуск,Параметр отпуск новый V
0,709046,37Г2Ф,127.0,12.5,77.5,105.275544,27.775544,771.480857,66.00,100.78453,...,0.0288,0.08827,0.00007,0.00011,0.00006,0.0077,0.670405,829.051634,18.748483,10.457088
1,709046,37Г2Ф,127.0,12.5,70.1,105.275544,35.175544,1237.318911,50.10,100.78453,...,0.0288,0.08827,0.00007,0.00011,0.00006,0.0077,0.670405,826.474514,18.637728,10.346333
2,709046,37Г2Ф,127.0,12.5,70.1,105.275544,35.175544,1237.318911,50.10,100.78453,...,0.0288,0.08827,0.00007,0.00011,0.00006,0.0077,0.670405,823.567356,18.512164,10.220769
3,709046,37Г2Ф,127.0,12.5,70.1,105.275544,35.175544,1237.318911,50.10,100.78453,...,0.0288,0.08827,0.00007,0.00011,0.00006,0.0077,0.670405,820.230348,18.367211,10.075816
4,709046,37Г2Ф,127.0,12.5,70.1,105.275544,35.175544,1237.318911,50.10,100.78453,...,0.0288,0.08827,0.00007,0.00011,0.00006,0.0077,0.670405,820.230348,18.425809,10.107962
0 исторический,804141,37Г2Ф,127.0,12.5,76.0,NaN,NaN,NaN,59.25,NaN,...,0.0200,0.08000,0.00000,0.00000,0.00000,0.0110,0.667000,835.526928,18.891510,10.384704


In [ ]:
# df_dict = pd.read_excel('DATA/modes_before_model/top_typesize(рекомендации).xlsx', sheetname=None)
# for sheet, df in zip(df_dict, df_dict.values()):
#     valid = clean_data_sheets(df, ls_need_col+['№ партии', 'марка стали'])
#     print(valid)

In [32]:
y_valid = tmp[targets[0]]
y_pred = tmp['predicted '+targets[0]]

from sklearn.metrics import r2_score 
from sklearn.metrics import explained_variance_score 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
print('corr',np.corrcoef(y_valid, y_pred)[0][1])
print('r2', r2_score(y_valid, y_pred))
print('explained_variance_score', explained_variance_score(y_valid, y_pred))
print('mean_squared_error', mean_squared_error(y_valid, y_pred))
print('mean_absolute_error', mean_absolute_error(y_valid, y_pred))
print('median_absolute_error', median_absolute_error(y_valid, y_pred))

d = {
    'corr': [np.corrcoef(y_valid, y_pred)[0][1]],
    'r2': [r2_score(y_valid, y_pred)],
    'explained_variance_score': [explained_variance_score(y_valid, y_pred)],
    'mean_squared_error': [mean_squared_error(y_valid, y_pred)],
    'mean_absolute_error': [mean_absolute_error(y_valid, y_pred)],
    'median_absolute_error': [median_absolute_error(y_valid, y_pred)]
}

pd.DataFrame(d).to_excel(output_filename+'_scores.xlsx')

corr 0.9491442637311868
r2 0.8930216852957163
explained_variance_score 0.8957239874637852
mean_squared_error 19.292150409068473
mean_absolute_error 3.2350262905404064
median_absolute_error 2.4166564587036916


In [31]:
y_valid = tmp[targets[1]]
y_pred = tmp['predicted '+targets[1]]

from sklearn.metrics import r2_score 
from sklearn.metrics import explained_variance_score 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
print('corr',np.corrcoef(y_valid, y_pred)[0][1])
print('r2', r2_score(y_valid, y_pred))
print('explained_variance_score', explained_variance_score(y_valid, y_pred))
print('mean_squared_error', mean_squared_error(y_valid, y_pred))
print('mean_absolute_error', mean_absolute_error(y_valid, y_pred))
print('median_absolute_error', median_absolute_error(y_valid, y_pred))

d = {
    'corr': [np.corrcoef(y_valid, y_pred)[0][1]],
    'r2': [r2_score(y_valid, y_pred)],
    'explained_variance_score': [explained_variance_score(y_valid, y_pred)],
    'mean_squared_error': [mean_squared_error(y_valid, y_pred)],
    'mean_absolute_error': [mean_absolute_error(y_valid, y_pred)],
    'median_absolute_error': [median_absolute_error(y_valid, y_pred)]
}

pd.DataFrame(d).to_excel(output_filename+'_scores.xlsx')

corr 0.9643158026543087
r2 0.9213201363358962
explained_variance_score 0.9219271770797535
mean_squared_error 11.392219003303019
mean_absolute_error 2.49501703374625
median_absolute_error 1.9771326530612328


In [93]:
for target in targets:
    print(
        df[df['MAE '+target]>5].shape[0]/df.shape[0],
          df[df['MAE '+target]>4].shape[0]/df.shape[0],
        df[df['MAE '+target]>3].shape[0]/df.shape[0])

0.20423000660938534 0.29610046265697293 0.4137475214805023
0.11103767349636484 0.18968935888962327 0.29808327825512226


In [17]:
d ={
   '% err >5': [df[df['MAE']>5].shape[0]/df.shape[0]],
   '% err >4':  [df[df['MAE']>4].shape[0]/df.shape[0]],
   '% err >3':  [df[df['MAE']>3].shape[0]/df.shape[0]]
}

pd.DataFrame(d).to_excel(dir_name+'/%.xlsx')

In [18]:
d

{'% err >3': [0.4527112232030265],
 '% err >4': [0.33921815889029006],
 '% err >5': [0.25598991172761665]}